# Ground State of a Molecule

Here are the necessary imports:

In [ ]:

from qiskit import Aer
from qiskit_nature.drivers import PySCFDriver, UnitsType, Molecule
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.algorithms import VQEUCCFactory
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.utils import QuantumInstance
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit.library import EfficientSU2, QAOAAnsatz

print("Libraries imported successfully")

## Defining the system

The first step is to define the molecular system. In the following we ask for the electronic part of a hydrogen molecule.

Qiskit is interfaced with different classical codes which are able to find the HF solutions. Interfacing between Qiskit and the following codes is already available: Gaussian, Psi4, PyQuante, PySCF.

In the following we set up a PySCF driver, for the hydrogen molecule at equilibrium bond length (0.735 angstrom) in the singlet state and with no charge.

In [ ]:
#Block 1 - Specifying the parameters of the molecule
molecule = Molecule(geometry=[['H', [0., 0., 0.]],
                              ['H', [0., 0., 0.01]]],
                     charge=0, multiplicity=1)
structure_problem = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM)
qubit_converter = QubitConverter(JordanWignerMapper())

## NumPyMinimumEigensolver

A solver is the algorithm through which the ground state is computed.

Let’s first start with a purely classical example: the NumPy minimum eigensolver. This algorithm exactly diagonalizes the Hamiltonian. Although it scales badly, it can be used on small systems to check the results of the quantum algorithms.

In [ ]:
#Block 2 - Solving using classical solver
method = NumPyMinimumEigensolver()

calc = GroundStateEigensolver(qubit_converter, method)
result = calc.solve(ElectronicStructureProblem(structure_problem))
print("Total energy = ", result.total_energies)
print("Electronic ground state energy =", result.computed_energies)
print("Nuclear repulsion energy =", result.nuclear_repulsion_energy)

## VQE Solver

In [ ]:
#Block 3 - Setting up VQE
qinstance = QuantumInstance(backend=Aer.get_backend('statevector_simulator'))

#tl_circuit = TwoLocal(rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz',
                     #entanglement='full', reps=1)
tl_circuit = EfficientSU2(reps= 1, entanglement='full')

method = VQE(ansatz = tl_circuit, quantum_instance = qinstance)


In [ ]:
#Block 4 - Solving using VQE
calc = GroundStateEigensolver(qubit_converter, method)
result = calc.solve(ElectronicStructureProblem(structure_problem))

print("Total energy = ", result.total_energies)
print("Electronic ground state energy =", result.computed_energies)
print("Nuclear repulsion energy =", result.nuclear_repulsion_energy)

Let's see the ansatz circuit used in the VQE solver:

In [ ]:
#Block 5 - Printing tunable circuit
ckt = tl_circuit.decompose()
ckt.draw()

In [ ]:
#Block 6 - Creating energy diagram using classical solver
method = NumPyMinimumEigensolver()

calc = GroundStateEigensolver(qubit_converter, method)
classical_energies = []
distances = [0.1,0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,1.9,2.1,2.3,2.5]

for i in range(13):
    # Get the distance
    dist = 
    # Define the molecule
    molecule = 
    
    # Define the structure problem 
    structure_problem = 
    
    result = calc.solve(ElectronicStructureProblem(structure_problem))
    classical_energies.append(result.total_energies)

In [ ]:
#Block 7 - Plotting classical energy diagram

plt.plot(distances, classical_energies, label="Classical Energy of the molecule")
plt.xlabel('Distance between atoms (Angstrom)')
plt.ylabel('Energy')
plt.legend()
plt.show()

In [ ]:
#Block 8 - Obtaining VQE energy diagram
qinstance = QuantumInstance(backend = Aer.get_backend('statevector_simulator'))
tl_circuit = TwoLocal(rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz',
                     entanglement='full', reps=3)
method = VQE(ansatz = tl_circuit, quantum_instance = qinstance)
calc = GroundStateEigensolver(qubit_converter, method)

vqe_energies = []
distances = [0.1,0.3,0.5,0.7,0.9,1.1,1.3,1.5,1.7,1.9,2.1,2.3,2.5]

for i in range(13):
    # ADD HERE!
    
    #es_problem = ElectronicStructureProblem(driver)
    result = calc.solve(ElectronicStructureProblem(structure_problem))
    vqe_energies.append(result.total_energies)

In [ ]:
#Block 9 - Plotting VQE energy diagram
plt.plot(distances, classical_energies, distances, vqe_energies,marker = "o")
plt.xlabel('Distance between atoms (Angstrom)')
plt.ylabel('Energy')
plt.legend(['Classical Energy of the molecule','VQE energy of the molecule'])
plt.show()

## Running VQE on a real quantum computer

We will use real noise data for an IBM Quantum device using the date stored in Qiskit Terra. Specifically, the device is `ibmq_vigo`.

In [ ]:
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy

In [ ]:
IBMQ.load_account()
# Get the least busy backend
provider = IBMQ.get_provider(hub='ibm-q')
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 
                                       and not x.configuration().simulator 
                                       and x.status().operational==True))
print("least busy backend: ", backend)

In [ ]:
quantum_instance = QuantumInstance(backend = backend)
tl_circuit = TwoLocal(rotation_blocks = ['h', 'rx'], entanglement_blocks = 'cz',
                     entanglement='full', reps=3)
method = VQE(ansatz = tl_circuit, quantum_instance = quantum_instance)
calc = GroundStateEigensolver(qubit_converter, method)

res = calc.solve(ElectronicStructureProblem(structure_problem))
print(res)